

# 基于修改后的 MindQuantum 0.5.0 采用变分算法重构半导体双量子点单-三重态量子 Z 门

# 用 abs 的自定义 MQLayer, 初始化方式为全 1 初始化。

In [3]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_z_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_z_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_z_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_z_eval_y.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)

circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(0)
circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)


ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('mqvector', circ.n_qubits)
sim_left = Simulator('mqvector',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)
lr = 0.05
Quantum_net = MQLayer(grad_ops)
opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
net = TrainOneStepCell(Quantum_net, opti)
error_min = 1
for j in range(len(train_x)):
    net(Tensor(train_x[j]), Tensor(train_y[j]))
    params = abs(Quantum_net.weight.asnumpy())
    final_state = []
    for k in range(100): # 100 个测试点
        sim.reset()
        sim.set_qs(eval_x[k])
        sim.apply_circuit(circ, params)
        final_state.append(sim.get_qs())
    error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y)]))
    error_min = min(error, error_min)
    if j % 100 == 0:
        print('error_min:', error_min, 'error_now:', error, 'train_num:', j)
    if error_min < 1e-5:
        break
        
print('\nerror_min:', error_min, 'train_num:', j)
print('params:\n', params)

error_min: 0.36166873449762404 error_now: 0.36166873449762404 train_num: 0
error_min: 0.012605211232422198 error_now: 0.07491836759353754 train_num: 100
error_min: 0.002565028182023643 error_now: 0.03273801323739367 train_num: 200
error_min: 0.002565028182023643 error_now: 0.8495895412838386 train_num: 300
error_min: 0.002565028182023643 error_now: 0.7670285130720569 train_num: 400
error_min: 0.002565028182023643 error_now: 0.011949284981079944 train_num: 500
error_min: 6.752825516542771e-06 error_now: 6.752825516542771e-06 train_num: 600

error_min: 6.752825516542771e-06 train_num: 600
params:
 [1.6117827  1.3258268  0.0498501  2.3190951  0.27652496 1.5626596
 0.70895815 0.61659425 1.8357016  1.4761814  2.117067   0.6488876 ]
